In [1]:
import sys
import logging
import pandas as pd
import openai
import os
from dotenv import load_dotenv

sys.path.append('..')
import nature_go_client
from question_generation import generate_questions

logger = logging.getLogger(__name__)


load_dotenv()
NG_USERNAME = os.getenv("NG_USERNAME")
NG_PASSWORD = os.getenv("NG_PASSWORD")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
client = nature_go_client.NatureGoClient(username=NG_USERNAME, password=NG_PASSWORD)
client.login()

## Retrieve species with missing text content through Nature go API

In [3]:
species_list = pd.DataFrame(client.get_labeled_species(limit=12, multiplechoicequestions=False, descriptions=False))
# species_list = species_list[species_list['num_observations_total'] > 0]
species_list

,id,scientificNameWithoutAuthor,commonNames,genus,family,gbif_id,powo_id,wikipedia_word_count,number_of_occurrences,occurences_cdf,...,illustration,illustration_transparent,illustration_reference,illustration_reference_transparent,reference_image_url,rarity,illustration_url,display_name,num_observations_total,num_questions_total
0,36473,Rosa x odorata,"[Tea rose, Spice Humes blush]",Rosa,Rosaceae,,,NaN,NaN,NaN,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Legendary,http://nature-go.edouardleurent.com/media/spec...,Tea rose,4,0
1,83,Ilex aquifolium,"[Holly, English holly, Common holly]",Ilex,Aquifoliaceae,5414222.0,83051-1,1167.0,3430.0,0.745357,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,https://upload.wikimedia.org/wikipedia/commons...,Very Common,http://nature-go.edouardleurent.com/media/spec...,Holly,1,0
2,135,Laurus nobilis,"[(Sweet) Bay leaf, Bay Laurel, Sweet bay]",Laurus,Lauraceae,3034015.0,465049-1,1725.0,2566.0,0.649621,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Common,http://nature-go.edouardleurent.com/media/spec...,(Sweet) Bay leaf,1,0
3,229,Olea europaea,"[Olive, زیتون, Oliveleaf]",Olea,Oleaceae,5415040.0,610675-1,6854.0,1742.0,0.525610,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Very Common,http://nature-go.edouardleurent.com/media/spec...,Olive,1,0
4,2214,Pittosporum tenuifolium,"[Tawhiwhi, Kohuhu, Thin leaved pittosporum]",Pittosporum,Pittosporaceae,2986293.0,684708-1,1518.0,67.0,0.057402,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Uncommon,http://nature-go.edouardleurent.com/media/spec...,Tawhiwhi,1,0
5,2797,Tradescantia zebrina,"[Inchplant, Wandering-Jew, گل برگ بیدی]",Tradescantia,Commelinaceae,2765359.0,30049896-2,750.0,38.0,0.039018,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Very Common,http://nature-go.edouardleurent.com/media/spec...,Inchplant,1,0
6,7260,Rosa foetida,"[Austrian copper rose, Austrian Briar, Austria...",Rosa,Rosaceae,3006902.0,732502-1,357.0,3.0,0.006578,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Uncommon,http://nature-go.edouardleurent.com/media/spec...,Austrian copper rose,1,0
7,7857,Cephalotaxus fortunei,"[Fortune's plum-yew, Chinese plum-yew, Fortune...",Cephalotaxus,Taxaceae,2685265.0,261815-1,669.0,3.0,0.005480,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Rare,http://nature-go.edouardleurent.com/media/spec...,Fortune's plum-yew,1,0
8,14367,Phalaenopsis × singuliflora,"[Moth Orchid, Orchidea, ارکیده دو رنگ]",Phalaenopsis,Orchidaceae,5310894.0,650593-1,59.0,-1.0,0.000000,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Very Common,http://nature-go.edouardleurent.com/media/spec...,Moth Orchid,1,0
9,36471,Centranthus lecoqii,"[Red Valerian, Narrow-leaved Valerian, Common ...",Centranthus,Caprifoliaceae,,,NaN,NaN,NaN,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Legendary,http://nature-go.edouardleurent.com/media/spec...,Red Valerian,1,0


In [4]:
species = species_list.iloc[-1]
print(species)

id                                                                                    1
scientificNameWithoutAuthor                                              Prunus spinosa
commonNames                           [Blackthorn, Sloe, Sloe Berry, Sloe Of Europe,...
genus                                                                            Prunus
family                                                                         Rosaceae
gbif_id                                                                       3023221.0
powo_id                                                                        730297-1
wikipedia_word_count                                                             1269.0
number_of_occurrences                                                           10581.0
occurences_cdf                                                                 0.993512
rarity_gpt                                                                          3.8
descriptions                    

## Run generation with openai API

In [5]:
import summary_generation
import summary_prompt
import importlib
importlib.reload(summary_generation)
importlib.reload(summary_prompt)

summary = summary_generation.generate_summaries(common_name=species.display_name, scientific_name=f'{species.display_name} ({species.scientificNameWithoutAuthor})', material=None, prompt=summary_prompt.summary_v7)

In [6]:
print('############ Full ############')
print(summary['original'])
print('\n############ Part 1 ############')
print(summary['part_1'])
print('\n############ Part 2 ############')
print(summary['part_2'])
print('\n############ Part 3 ############')
print(summary['part_3'])

############ Full ############
Today I made a fascinating discovery while studying the blackthorn, also known as sloe. This shrub, with its spiny branches and dark bark, is native to Europe and parts of Asia and Africa. It has even made its way to places like New Zealand and the United States. The blackthorn is quite a versatile plant, with its fruits being used to make sloe gin in Britain and patxaran in the Basque Country. The wood of the blackthorn is also prized for making walking sticks, including the famous Irish shillelagh.

<BREAK_1>

As I examined the blackthorn more closely, I noticed its distinct features. It can grow up to 5 meters tall and has oval leaves with serrated edges. In early spring, before the leaves appear, the blackthorn produces beautiful creamy-white flowers. These flowers are pollinated by insects and give way to the small black fruits known as sloes. These sloes are quite tart and astringent, but they can be used to make preserves or even infused into gin o

In [7]:
import question_generation
import importlib
importlib.reload(question_generation)

material = None
material = ' '.join([summary['part_1'], summary['part_2'], summary['part_3']])
questions = question_generation.generate_questions(common_name=species.display_name, scientific_name=species.scientificNameWithoutAuthor, material=material)

In [8]:
questions

[{'question': "What is one use of the blackthorn's fruits?",
  'choices': ['Making sloe gin',
   'Creating patxaran',
   'Making preserves',
   'All of the above'],
  'correct_choice': 3},
 {'question': "What is a distinct feature of the blackthorn's flowers?",
  'choices': ['They are creamy white',
   'They are yellow or red',
   'They are pollinated by birds',
   'They give way to sloes'],
  'correct_choice': 0},
 {'question': 'How tall can the blackthorn grow?',
  'choices': ['Up to 3 meters',
   'Up to 5 meters',
   'Up to 7 meters',
   'Up to 10 meters'],
  'correct_choice': 1},
 {'question': 'What is one key difference between the blackthorn and the cherry plum?',
  'choices': ['The color of their flowers',
   'The size of their fruits',
   'The shape of their leaves',
   'The method of pollination'],
  'correct_choice': 0}]